In [366]:
# # Installing necessary dependencies
import sys
# !conda install --yes --prefix {sys.prefix} PIL
# # !{sys.executable} -m pip install ____

# !pip install openpyxl
# !{sys.executable} -m pip install pytesseract
# !{sys.executable} -m pip install pillow
# !{sys.executable} -m pip install python-poppler
# !{sys.executable} -m pip install opencv-python
# !{sys.executable} -m pip install --upgrade pip
# !conda install -c conda-forge poppler

# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install sklearn
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install nltk
# !{sys.executable} -m pip install pandas


In [80]:
# Checking which environment we are working in
# !conda info --envs # check which env we are in
# !conda activate HCP-env # Activate the right environment

In [78]:
# Below, I try to do some text processing and tokenization to see word frequencies

import codecs
import re
import copy
import collections

import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from __future__ import division
import matplotlib
import json


In [14]:
nltk.download('stopwords') # downloads all stopwords to avoid

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\15713\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.corpus import stopwords 

In [18]:
nltk.download('punkt') # downloads the tokenizer we need

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\15713\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
# Function to tokenize a text file into sentences or phrases
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def get_sentence_counter(path=None, text=None):
    if path:
        path = os.getcwd()
        for file in os.listdir():
            if file.endswith('.txt'):
                with codecs.open(file, 'r') as f: 
                    text = f.read()
    else:
        text = text
    raw_sentences = tokenizer.tokenize(text)
    sentences = []
    for raw_sentence in raw_sentences:
        tmp = raw_sentence.split('\n')
        for x in tmp:
            sentences.append(x)

    return sentences, len(sentences)

In [ ]:
################################################################################
################################################################################
################################################################################
# INDEED

In [242]:
# Working with 'Master_Indeed_Dataset'
path = os.path.join(os.getcwd(), 'Master_Indeed_Dataset.xlsx')
df = pd.read_excel(path)

In [243]:
df.head(6)

,Title,Company,Location,Links,Salary,Description
0,JUNIOR PENETRATION TESTER,Elevate,Remote,https://indeed.com/viewjob...,$35 - $50 an hour,JUNIOR PENETRATION TESTER\...
1,Junior Penetration Tester ...,BreachLock,Floridaâ€¢Remote,https://indeed.com/viewjob...,None,The Penetration Testing pr...
2,Red Team - Penetration Tester,"Simplicity, LLC.",Remote,https://indeed.com/viewjob...,"$75,000 - $90,000 a year",Who are we?\nBreachLock is...
3,Penetration Tester,Creative Visions,"Washington, DCâ€¢Remote",https://indeed.com/viewjob...,None,"Fort Mill, SC, Carrollton ..."
4,Penetration Tester,TechTrueUP,Remote,https://indeed.com/viewjob...,$45 - $50 an hour,Summary:\nThe Penetration ...
5,Penetration Tester (remote),Fast AF,"San Francisco, CAâ€¢Remote",https://indeed.com/viewjob...,None,Job Details:\nIn this vita...


In [244]:
# Creating a dataframe with title as index and description as an attribute
title, description = df['Title'].to_list(), df['Description'].to_list()
indeed_df = pd.DataFrame(data={"Title": title,
                               "Description": description} 
                         )
# removing rows with 'None' for their Job-Title
indeed_df = indeed_df[indeed_df.Title != 'None']

In [245]:
indeed_df.head(3)

,Title,Description
0,JUNIOR PENETRATION TESTER,JUNIOR PENETRATION TESTER\...
1,Junior Penetration Tester ...,The Penetration Testing pr...
2,Red Team - Penetration Tester,Who are we?\nBreachLock is...


In [246]:
# Converting the above dataframe into dictionary for iteration {title: description}
indeed = {}
for (t, d) in zip(title, description): # loops through the list title and description
    if t in indeed and t!='None': # if the job title already exists, we lump the descriptions together
        indeed[t]+=d
    else:
        indeed[t] = d # sets the title as key and the description as the value


In [247]:
# iterating through the Job's to tokenize into sentences/phrases
indeed_tokenized_dict = {}
for key in indeed:
    if key != 'None':
        text = indeed[key]
        # Tokenize into sentence or phrase
        sentence_count, s1 = get_sentence_counter(text=text)
        indeed_tokenized_dict[key] = sentence_count


In [248]:
# Putting in dataframe with keys/job titles as titles and the values as the tokenized sentences just for visual purposes
df_concat = pd.DataFrame()
for k,v in indeed_tokenized_dict.items():
    if k != 'None':
        df = pd.DataFrame({k: v})
        df_concat = pd.concat([df_concat, df], axis=1)

df_concat.T.head(3)

,0,1,2,3,4,5,6,7,8,9,...,1445,1446,1447,1448,1449,1450,1451,1452,1453,1454
JUNIOR PENETRATION TESTER,JUNIOR PENETRATION TESTER,www.elevateconsult.com,ABOUT ELEVATE,Elevate is a premier consu...,Our team consists of self-...,We value exceptional clien...,We are looking for a motiv...,ABOUT THE POSITION,This is a contract to-hire...,Elevate is looking for a J...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Junior Penetration Tester USA Remote,The Penetration Testing pr...,The consultant would perfo...,The assessments that would...,For the remote assessments...,The consultant should be c...,The consultant should have...,Must Haves:,Background in Enterprise n...,1-3 years experience perfo...,3-5 years of experience in...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Red Team - Penetration Tester,Who are we?,BreachLock is a security s...,The BreachLock platform le...,BreachLocksâ€™s modern Saa...,"As a result, CIOâ€™s and C...",The BreachLock platform fa...,Some of our achievements i...,One of the fastest-growing...,Cyber Security Innovator f...,Top 10 Vulnerability Manag...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [338]:
# shows the description
# indeed['SOC Analyst']

In [337]:
# shows the sentences in the description tokenized
# indeed_tokenized_dict['SOC Analyst']

In [315]:
# Here I will try to create 'bag of words' that could be used to identify if a sentence is refering to 
# ability, skill, knowledge or other
bag_A = [' able ', ' ability', 'Ability ', 'abilities', 'perform']
bag_S = ['skill', 'Skill',]
bag_K = ['know', 'known', 'knowledge', 'Knowledge']

# Helper function to check whether the flag words are present in the sentence
def check_word(word_list,sentence):
    for x in word_list:
        return True if x in sentence else False

In [316]:
# Now I will sort out the sentences into Ability, skill, knowledge or other and save it as a nested dict
Master_indeed_dict = {}
K,S,A, other = [],[],[], []
for k in indeed_tokenized_dict.keys():
    for sentence in indeed_tokenized_dict[k]:
        if check_word(bag_K,sentence):
            K.append(sentence)
        elif check_word(bag_S,sentence):
            S.append(sentence)
        elif check_word(bag_A,sentence):
            A.append(sentence)
        else:
            other.append(sentence)
            
    Master_indeed_dict[k] = {'K':K,
                             'S':S,
                             'A':A,
                            'other': other}

    

In [369]:
Master_indeed_dict['JUNIOR PENETRATION TESTER']['K']

['The Junior Penetration Tester will support assessments of clients where the primary goal is to penetrate our clientâ€™s network, applications, and/or operating systems by using your advanced knowledge of manipulation and exploitation of misconfigurations.',
 'Demonstrated knowledge of vulnerability and penetration testing processes, Experience with Commercial tools Burp Qualys, Metasploit, Cain and Abel.',
 'Ability to utilize technical tools (online and otherwise), analyze the results of scans and other technical reports and the ability to research and expand personal technical knowledge when needed',
 'Intermediate to advance knowledge of various Penetration testing frameworks',
 'Intermediate to advance knowledge of networking operations such as WAFs, Firewalls, and Transport layer Protocols,',
 'Intermediate to advance knowledge of programming languages such as C, Java, .MS SQL, SQLi and ability to code review',
 'Do you have intermediate to advanced knowledge of any of the follo

In [370]:
Master_indeed_dict['JUNIOR PENETRATION TESTER']['S']

['Our team consists of self-managed, high-caliber professionals with deep skills in the Cybersecurity IT Governance, Risk and Controls.',
 'Elevate is looking for a Junior Penetration Tester (â€œPenTest Consultantâ€\x9d) who possesses a broad skillset, demonstrating excellent communication and presentations skills, analytical thinking, and a desire to learn and contribute to our Cybersecurity team.',
 'We seek individuals who enjoy learning to add to their skillset to quickly problem-solve tough solutions.',
 'Transparent reporting which can include a forecasted schedule submitted weekly, accurate and daily time reporting and project budget reporting and managing skills.',
 'The candidate must have excellent verbal and written communications skills in order to be able to convey the risk and impact of the findings to technical and non-technical stakeholders.',
 'Excellent client service and project management skills (i.e.',
 'Strong analytical and communications (written and oral) skill

In [371]:
Master_indeed_dict['JUNIOR PENETRATION TESTER']['A']

['Our ideal candidate will be able to think and work creatively, both in teams and independently.',
 'Must be able to understand client needs, seek clarification and perform independent research',
 'Be a strong, confident, and exacting writer and speaker, able to communicate your vision and roadmap effectively to a wide variety of stakeholders',
 'Individual should have a mindset of a defender and should be able to operate in a fast-paced environment working closely with our support teams which includes Network, Firewall, Server and Application development teams.',
 'Â· Be able to handle the most dynamic and challenging scenarios encountered by the program and assist other analysts by providing direction and guidance when necessary.',
 'Candidate will need to be able to perform assessment on various system including obtaining evidence and writing final assessment reports',
 'Â· Candidate should be able to perform network, application, system, and mobile penetration testing across the p

In [372]:
Master_indeed_dict['JUNIOR PENETRATION TESTER']['other']

['JUNIOR PENETRATION TESTER',
 'www.elevateconsult.com',
 'ABOUT ELEVATE',
 'Elevate is a premier consulting firm specialized in Cyber Security, IT Compliance, Privacy, Internal Audit and Process Improvement.',
 'We value exceptional client service, solving and coaching our clients through complex and critical transformations and providing effective communication (both verbal & written).',
 'We are looking for a motivated, experienced and self-managed Junior Penetration Tester to join our growing Cybersecurity team.',
 'ABOUT THE POSITION',
 'This is a contract to-hire-position which starts at a competitive hourly rate with the opportunity to evolve into a full-time position with an annual salary and benefits for the right candidate.',
 'The role interacts frequently with clients, engagement team, and the firmâ€™s leadership team.',
 'RESPONSIBILITIES AND QUALIFICATIONS',
 'Required General Skills/Knowledge',
 'Familiarity and wiliness to learn how to perform white and grey box penetra

In [324]:
# Exporting dictionary as a json file for use during sentence vectorization
data = Master_indeed_dict
with open('Master_indeed_dict.json', 'w') as f:
    json.dump(data, f)

In [ ]:
################################################################################
################################################################################
################################################################################
# SYLLABI

In [348]:
#working with the syllabi data from NICCS
path = os.path.join(os.getcwd(), 'syllabi_data_niccs.csv')
NICCS = pd.read_csv(path, encoding='cp1252')

In [350]:
NICCS.head(5)

,College or Institution,City,State,Deg,AreaofStudy,Classroom,Online,CourseTitle,Course Description,Learning Objectives,FrameworkConnections,ConnectwithSpecialtyAreas,WorkRole,WorkRoleID,WorkRoleDesc,Abilities,Knowledge,Skills,Link
0,Kennesaw State University,Kennesaw,GA,BS,Cybersecurity,1,1,Network Configuration & Ad...,Topics include design and ...,Demonstrate how to manage ...,Operate and Maintain;,Network Security,Network Operations Specialist,OM-NET-001,"Plans, implement, operates...",A0052: Ability to operate ...,K0001: Knowledge of comput...,S0004: Skill in analyzing ...,https://niccs.cisa.gov/tra...
1,Kennesaw State University,Kennesaw,GA,BS,Cybersecurity,1,1,Wireless Security,Topics include Security an...,Describe the differences b...,Operate and Maintain,Network Security,Network Operations Specialist,OM-NET-001,"Plans, implement, operates...",A0052: Ability to operate ...,K0001: Knowledge of comput...,S0004: Skill in analyzing ...,https://niccs.cisa.gov/tra...
2,Kennesaw State University,Kennesaw,GA,BS,Cybersecurity,1,1,Wireless Security,Topics include Security an...,Describe the differences b...,Oversee and Govern,Cybersecurity Management,Communications Security (C...,OV-MGT-002,Individual who manages com...,A0162: Ability to recogniz...,K0001: Knowledge of comput...,S0027: Skill in determinin...,https://niccs.cisa.gov/tra...
3,Kennesaw State University,Kennesaw,GA,BS,Cybersecurity,1,1,Ethical Hacking for Effect...,This course focuses on det...,Identify what an ethical h...,Analyze; Operate and Maint...,Cyberdefense Analysis,Cyber Defense Analyst,PR-CDA-001,Uses data collected from a...,A0010: Ability to analyze ...,K0001: Knowledge of comput...,S0020: Skill in developing...,https://niccs.cisa.gov/tra...
4,Kennesaw State University,Kennesaw,GA,BS,Cybersecurity,1,1,Ethical Hacking for Effect...,This course focuses on det...,Identify what an ethical h...,Analyze; Operate and Maint...,Explotaition Analysis,Explotaition Analysis,AN-EXP-001,Collaborates to identify a...,A0013: Ability to communic...,K0001: Knowledge of comput...,S0066: Skill in identifyin...,https://niccs.cisa.gov/tra...


In [351]:
# Converting Columns needed to list for easier itteration
CourseTitle = NICCS['CourseTitle'].to_list()
WorkRole = NICCS['WorkRole'].to_list()
WorkRoleID = NICCS['WorkRoleID'].to_list()
Tasks = NICCS['WorkRoleDesc'].to_list()
Knowledge = NICCS['Knowledge'].to_list()
Skill = NICCS['Skills'].to_list()
Ability = NICCS['Abilities'].to_list()

In [362]:
# Saving each KSA under each work role as a nested dictionary
Syllabi_dict = {}
pattern = '[S|A|K][0-9]+[\S][\s]'
for i,WR in enumerate(WorkRole):
    if WR not in Syllabi_dict.keys(): # Avoiding duplicates
        # splitting to get each KSA & IDs, while also removing the A####,K####, S#### tags
        K = re.sub(pattern, '', Knowledge[i]).split(";") 
        S = re.sub(pattern, '', Skill[i]).split(";")
        A = re.sub(pattern, '', Ability[i]).split(";")
        Syllabi_dict[WR] = {'ID': WorkRoleID[i], 
                             'K': K, 
                             'S': S, 
                             'A': A} #Saving as a nested dict

In [375]:
Master_indeed_dict['Entry-Level Cyber Defense Analyst']['S']

['Our team consists of self-managed, high-caliber professionals with deep skills in the Cybersecurity IT Governance, Risk and Controls.',
 'Elevate is looking for a Junior Penetration Tester (â€œPenTest Consultantâ€\x9d) who possesses a broad skillset, demonstrating excellent communication and presentations skills, analytical thinking, and a desire to learn and contribute to our Cybersecurity team.',
 'We seek individuals who enjoy learning to add to their skillset to quickly problem-solve tough solutions.',
 'Transparent reporting which can include a forecasted schedule submitted weekly, accurate and daily time reporting and project budget reporting and managing skills.',
 'The candidate must have excellent verbal and written communications skills in order to be able to convey the risk and impact of the findings to technical and non-technical stakeholders.',
 'Excellent client service and project management skills (i.e.',
 'Strong analytical and communications (written and oral) skill

In [376]:
Syllabi_dict['Cyber Defense Analyst']['S']

['Skill in developing and deploying signatures',
 ' Skill in detecting host and network based intrusions via intrusion detection technologies (e.g., Snort)',
 ' Skill in determining how a security system should work (including its resilience and dependability capabilities) and how changes in conditions, operations, or the environment will affect these outcomes',
 ' Skill in evaluating the adequacy of security designs',
 ' Skill in using incident handling methodologies',
 ' Skill in using protocol analyzers',
 ' Skill in collecting data from a variety of cyber defense resources',
 ' Skill in recognizing and categorizing types of vulnerabilities and associated attacks',
 ' Skill in reading and interpreting signatures (e.g., snort)',
 ' Skill in assessing security controls based on cybersecurity principles and tenets. (e.g., CIS CSC, NIST SP 800-53, Cybersecurity Framework, etc.)',
 ' Skill in performing packet-level analysis',
 ' Skill in recognizing vulnerabilities in security systems. 

In [367]:
# Exporting dictionary as a json file for use during sentence vectorization
data = Syllabi_dict
with open('syllabi_NICCS.json', 'w') as f:
    json.dump(data, f)

In [ ]:
################################################################################
################################################################################
################################################################################
# Certs

In [161]:
#working with the Certificate data from NICCS
path = os.path.join(os.getcwd(),'cert_list.xlsx' )
Certs = pd.read_excel(path) 

In [234]:
Certs.head(5)

,Cert Name,Cost,Time (hrs),Avg Salary of Recipient,Number of Holders,Knowledge,Skills,Abilities
0,CISSP (Certified Informati...,699,40-50,122138.0,92976.0,K0001: Knowledge of comput...,S0011: Skill in conducting...,A0002: Ability to match th...
1,CISA (Certified Informatio...,595,112,132278.0,150000.0,K0001: Knowledge of comput...,S0027: Skill in determinin...,A0162: Ability to recogniz...
2,CISM (Certified Informatio...,760,40-50,148622.0,50000.0,K0001: Knowledge of comput...,S0027: Skill in determinin...,A0162: Ability to recogniz...
3,Security+,370,45,72244.0,500000.0,K0001: Knowledge of comput...,S0020: Skill in developing...,A0010: Ability to analyze ...
4,CEH (Certified Ethical Hac...,1199,70-75,104813.0,79617.0,K0001: Knowledge of comput...,S0005: Skill in applying a...,A0008: Ability to apply th...


In [185]:
cert_name = Certs['Cert Name'].to_list()
c_K = Certs['Knowledge'].to_list()
c_S = Certs['Skills'].to_list()
c_A = Certs['Abilities '].to_list()

In [237]:
# Saving each KSA under each Cert as a nested dictionary
Cert_dict = {}
pattern = '[S|A|K][0-9]+[\S][\s]'
for i,cert in enumerate(cert_name):
    if cert not in Cert_dict.keys(): # Avoiding duplicates
        K = 'None' if pd.isna(c_K[i]) else re.sub(pattern, '', c_K[i]).split("\n")
        S = 'None' if pd.isna(c_S[i]) else re.sub(pattern, '', c_S[i]).split("\n")
        A = 'None' if pd.isna(c_A[i]) else re.sub(pattern, '', c_A[i]).split("\n")

        Cert_dict[cert] = {'K': K, 
                           'S': S, 
                           'A': A} #Saving as a nested dict

In [239]:
Cert_dict.keys()

dict_keys(['CISSP (Certified Information Systems Security Professional)', 'CISA (Certified Information Systems Auditor)', 'CISM (Certified Information Security Manager)', 'Security+', 'CEH (Certified Ethical Hacker)', 'GSEC (GIAC Security Essentials Certification)', 'SSCP (Systems Security Certified Practitioner)', 'CASP (CompTIA Advanced Security Practitioner)', 'GCIH (GIAC Certified Incident Handler)', 'OSCP (Offensive Security Professional)', 'CRISC (Certified in Risk and Information Systems Control)', 'AWS Certified Security', 'CCSP (Certified Cloud Security Professional)', 'CySA (Cybersecurity Analyst)', 'PCSE (Google Cloud Certified, Professional Cloud Security Engineer) ', 'HCISPP (HealthCare Information Security and Privacy Practitioner)', 'CopmTIA Pen Test+', 'CCSK (Certificate of Cloud Security Knowledge)', 'ECSA (EC-Certified Security Analyst)', 'GPEN (GIAC Penetration Tester)', 'CCFP (Certified Cyber Forensics Professional)', 'CAP (Certified Authorized Professionals)', 'CCS

In [264]:
Cert_dict['CISA (Certified Information Systems Auditor)']['A']

['Ability to recognize the unique aspects of the Communications Security (COMSEC) environment and hierarchy. ',
 'Ability to interpret Communications Security (COMSEC) terminology, guidelines and procedures. ',
 'Ability to identify the roles and responsibilities for appointed Communications Security (COMSEC) personnel. ',
 'Ability to manage Communications Security (COMSEC) material accounting, control and use procedure. ',
 'Ability to identify types of Communications Security (COMSEC) Incidents and how they’re reported ',
 'Ability to recognize the importance of auditing Communications Security (COMSEC) material and accounts. ',
 'Ability to Identify the requirements of In-Process accounting for Communications Security (COMSEC) ',
 'Ability to recognize the unique aspects of the Communications Security (COMSEC) environment and hierarchy.']

In [238]:
# Exporting dictionary as a json file for use during sentence vectorization
data = Cert_dict
with open('Certs.json', 'w') as f:
    json.dump(data, f)